# Attempting to set up SMIRNOFF simulation of host-guest complex

(Author: David Mobley, UCI)

Slochower's desired starting point is to use the files in the `original` directory to set up a SMIRNOFF host-guest simulation. Specifically: 
> This brings me to the second point — determining the input and output for the transformation. As you know, there are several parallel pathways we can follow. I’ve been thinking a lot about this. I think the most useful starting point would be a solvated prmtop and inpcrd with GAFF atom types. As far as I know this should be okay, since Chris Bayly was confident OpenEye tools can read GAFF atom types. I, however, have not gotten this to work myself. If the solvated files don’t work, the next best thing is probably separate mol2 files for the host and guest with SYBYL atom types.

So let's start by trying that.

## Attempt to convert/read solvated AMBER format files and create SMIRNOFF system

### A key first step is to get OEMols of the components to use with the Topology (which is already available)

There should be several ways to get OEMols of the components, in all likelihood. A couple come to mind:
- Convert the system into a PDB then load the components as OEMols:
    - Use AMBERTools (or AMBER) to convert the inpcrd to a PDB; use OpenEye tools to read the PDB file and split into components
    - Use ParmEd to convert the system to a PDB; use OpenEye tools to read the PDB file and split into components
    - Use MDTraj to convert to a PDB; use OpenEye tools to read the PDB file and split into components
    - Use OpenMM to load and write to a PDB via PDBFile
- Use ParmEd to load the system and split into components then generate topologies for the components; use oeommtools.utils to generate OEMols for the components

I seem not to have `ambpdb` installed; it must not be a component of AMBERTools. So I'll bypass that option for now and try some of the others. Let's attempt mdtraj first.

In [5]:
# Load trajectory via mdtraj
import mdtraj
import os
traj = mdtraj.load('original/solvated.inpcrd', top='original/solvated.prmtop')

# Save PDB
outdir = 'mobley_generated'
if not os.path.isdir(outdir): os.mkdir(outdir)
traj.save_pdb(os.path.join(outdir, 'solvated.pdb'))

In [ ]:
# Load PDB file and split into components

# Import oeommtools which has a useful splitter that wraps other openeye functions
import oeommtools.utils as oeutils



Stopping here after conversation with Slochower; he has systems running so main problem at present is actually the file size issue where every water molecule becomes a unique entry when converting to AMBER format.


## Check water/system size issue on conversion to AMBER

Let's load a system containing water and convert to AMBER format and see what happens to file sizes.

There is a pretty extensive example in the tests, i.e. https://github.com/openforcefield/openforcefield/blob/master/openforcefield/tests/test_smirnoff.py#L703 -- looks at system solvated with TIP3P. But I think that's with one molecule of TIP3P.

There's also a `cyclohexane_water` example (one molecule of cyclohexane in water) looked at in the tests).

In [12]:
from openforcefield.typing.engines.smirnoff import *
from simtk.openmm.app import PDBFile

# Load PDB of full system
filename = get_data_filename(os.path.join('systems', 'packmol_boxes', 'cyclohexane_water.pdb'))
pdbfile = PDBFile(filename)

# Load files of monomers
monomers = ['water', 'cyclohexane']
mols = list()
from openeye import oechem
mol = oechem.OEGraphMol()
for monomer in monomers:
    filename = get_data_filename(os.path.join('systems', 'monomers', monomer + '.sdf'))
    ifs = oechem.oemolistream(filename)
    while oechem.OEReadMolecule(ifs, mol):
        oechem.OETriposAtomNames(mol)
        mols.append( oechem.OEGraphMol(mol) )


# Load FF
ff = ForceField(get_data_filename('forcefield/smirnoff99Frosst.ffxml'))

## Create system
system = ff.createSystem( pdbfile.topology, mols)

In [14]:
import parmed as pmd
structure = pmd.openmm.topsystem.load_topology(pdbfile.topology, system, pdbfile.positions)
structure.save(os.path.join(outdir, 'solvated_smirnoff.prmtop'))


The file which results here is 396 MB, so this reproduces D. Slochower's issue where AMBER file formats are too large. I think this is probably due to issues with how molecules are mapped to one another - we're probably mapping water molecules in two different ways and thus ending up with very large systems as a result.

## What if we set up water using "normal" TIP3P and the rest of the system using SMIRNOFF?

There is an example of this, for setting up a normal protein and merging with a SMIRNOFF ligand, in the `openforcefield` repo at: https://github.com/openforcefield/openforcefield/blob/master/examples/mixedFF_structure/generate_mixedFF_complex.py

I think you should be able to achieve this by taking a solvated system and loading it into ParmEd, then:
- Using ParmEd's splitting functionality to split it up into its pieces
- Take just the pieces you want (water) into a separate system, (A), where you rejoin them
- Take the other pieces you want (host+guest, or solute) into a separate system, (B), where you rejoin them
- Parameterize (A) in the normal way for AMBER or an OpenMM setup
- Parameterize (B) as per SMIRNOFF
- Rejoin as done in the setup example linked just above

An example of using ParmEd to split and merge can be found here: https://github.com/choderalab/openmoltools/blob/master/openmoltools/gromacs.py#L153-L177 -- see the `top.split()` to get components, then the `newtop` construction on lines 174-177. You would want to swap in your original positions, of course, but the positions are the easy part.

If you want things to look roughly like the `generate_mixedFF_complex` example just linked, you would use [`generateSMIRNOFFStructure`](https://github.com/openforcefield/openforcefield/blob/master/openforcefield/utils/utils.py#L52) on an OEMol of each non-water component of your system to get a parameterized molecule, then join the molecules (and the solvent) using ParmeEd afterwards.

Note that `generateSMIRNOFFStructure` uses the somewhat-deprecated `generateTopologyFromOEMol` functionality so it's possible this could be less than ideal for hosts (though you probably already know if it is or not); you might be better off doing basically the same thing using the related functgionality in `oeommtools.utils` (forget the exact name, but something like `OEMol_to_OpenMMTopology`.